![sslogo](https://github.com/stratascratch/stratascratch.github.io/raw/master/assets/sslogo.jpg)

#### The make_df helper function will be used to generate DataFrames

In [2]:
import pandas as pd
import numpy as np

In [3]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

#### For results requiring multiple tables, use the display class to format your results

In [5]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

# Combining Datasets: Concat and Append

## Simple Concatenation with ``pd.concat``

In [6]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])

#### Concatenate ser1 and ser2

- Concat takes in a list
- merge takes in dataframes directly

In [8]:
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [9]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
df3 = make_df('CD', [5, 6])

display('df1', 'df2', 'df3')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,C,D
5,C5,D5
6,C6,D6


#### Concatenate df1's and df2's rows

In [10]:
pd.concat([df1, df2])

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


#### Concatenate df1's and df2's columns

In [11]:
pd.concat([df1, df2], axis=1)

,A,B,A,B
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,NaN,NaN,A3,B3
4,NaN,NaN,A4,B4


### Duplicate indices

#### The following concatenation creates a table with repeated indexes. This is an issue that should always be avoided!

In [14]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


#### Cause the concatenation of x and y to throw an error with the verify_integrity parameter and catch the error. Print a fitting error message

In [18]:
try:
    pd.concat([x,y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


#### Concatenate x and y. Avoid an overlapping index error by creating a multiindex key

In [20]:
pd.concat([x,y], keys=['x','y'])

A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

#### Concatenate x and y. Avoid an overlapping index error by ignoring the original indexes

In [21]:
pd.concat([x,y], ignore_index=True)

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


### Concatenation with joins

#### The statements below show the default behavior of pd.concat on DataFrames. What type of join is the default? Add a join parameter with the default join type

In [24]:
df5 = make_df('ABC', [1, 2]) 
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6], join = "outer", sort=False)')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join = "outer", sort=False)
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

#### Perform the above concatenation using the append method

Remember: the append method is the specific case (axis=0, join='outer') of the concat method, the parameters cannot be modified

In [25]:
df5.append(df6)

/var/folders/wg/8yz1tny10yl9ct8bq7zl3pp00000gn/T/ipykernel_2725/551904218.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df5.append(df6)


,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


#### Concatenate df5 and df6 only keeping the columns from both tables

In [26]:
pd.concat([df5, df6], join = 'inner')

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


# Combining Datasets: Merge and Join

#### The worker, title, and bonus tables are from a relational database. 

In [27]:
worker = pd.read_csv('../datasets/worker.csv')
title = pd.read_csv('../datasets/title.csv')
bonus = pd.read_csv('../datasets/bonus.csv')
display('worker', 'title', 'bonus')

worker
   worker_id first_name last_name  salary        joining_date department
0          1     Monika     Arora  100000  2014-02-20 9:00:00         HR
1          2   Niharika     Verma   80000  2014-06-11 9:00:00      Admin
2          3     Vishal   Singhal  300000  2014-02-20 9:00:00         HR
3          4     Amitah     Singh  500000  2014-02-20 9:00:00      Admin
4          5      Vivek     Bhati  500000  2014-06-11 9:00:00      Admin
5          6      Vipul     Diwan  200000  2014-06-11 9:00:00    Account
6          7     Satish     Kumar   75000  2014-01-20 9:00:00    Account
7          8    Geetika   Chauhan   90000  2014-04-11 9:00:00      Admin

title
   worker_ref_id   worker_title       affected_from
0              1        Manager  2016-02-20 0:00:00
1              2      Executive  2016-06-11 0:00:00
2              8      Executive  2016-06-11 0:00:00
3              5        Manager  2016-06-11 0:00:00
4              4  Asst. Manager  2016-06-11 0:00:00
5              7      Executive  2016-06-11 0:00:00
6              6           Lead  2016-06-11 0:00:00
7              3           Lead  2016-06-11 0:00:00

bonus
   worker_ref_id  bonus_amount          bonus_date
0              1          5000  2020-02-16 0:00:00
1              2          3000  2011-06-16 0:00:00
2              3          4000  2020-02-16 0:00:00
3              1          4500  2020-02-16 0:00:00
4              2          3500  2011-06-16 0:00:00

#### The employee DataFrames are part of a seperate relational database

In [28]:
employee_group = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
employee_hire_date = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('employee_group', 'employee_hire_date')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


### One-to-one joins

#### Merge the employee_group and employee_hire_data tables

In [35]:
employee_group.merge(employee_hire_date)

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


#### Merge the worker and title tables

In [32]:
worker.merge(title, left_on = 'worker_id', right_on = 'worker_ref_id')

,worker_id,first_name,last_name,salary,joining_date,department,worker_ref_id,worker_title,affected_from
0,1,Monika,Arora,100000,2014-02-20 9:00:00,HR,1,Manager,2016-02-20 0:00:00
1,2,Niharika,Verma,80000,2014-06-11 9:00:00,Admin,2,Executive,2016-06-11 0:00:00
2,3,Vishal,Singhal,300000,2014-02-20 9:00:00,HR,3,Lead,2016-06-11 0:00:00
3,4,Amitah,Singh,500000,2014-02-20 9:00:00,Admin,4,Asst. Manager,2016-06-11 0:00:00
4,5,Vivek,Bhati,500000,2014-06-11 9:00:00,Admin,5,Manager,2016-06-11 0:00:00
5,6,Vipul,Diwan,200000,2014-06-11 9:00:00,Account,6,Lead,2016-06-11 0:00:00
6,7,Satish,Kumar,75000,2014-01-20 9:00:00,Account,7,Executive,2016-06-11 0:00:00
7,8,Geetika,Chauhan,90000,2014-04-11 9:00:00,Admin,8,Executive,2016-06-11 0:00:00


### Many-to-one joins

In [33]:
employee_supervisor = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                                    'supervisor': ['Carly', 'Guido', 'Steve']})
employee_supervisor

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


#### Merge the employee_group and employee_supervisor tables

In [34]:
employee_group.merge(employee_supervisor)

,employee,group,supervisor
0,Bob,Accounting,Carly
1,Jake,Engineering,Guido
2,Lisa,Engineering,Guido
3,Sue,HR,Steve


#### Merge the worker and bonus tables.

In [36]:
worker.merge(bonus, left_on = 'worker_id', right_on = 'worker_ref_id')

,worker_id,first_name,last_name,salary,joining_date,department,worker_ref_id,bonus_amount,bonus_date
0,1,Monika,Arora,100000,2014-02-20 9:00:00,HR,1,5000,2020-02-16 0:00:00
1,1,Monika,Arora,100000,2014-02-20 9:00:00,HR,1,4500,2020-02-16 0:00:00
2,2,Niharika,Verma,80000,2014-06-11 9:00:00,Admin,2,3000,2011-06-16 0:00:00
3,2,Niharika,Verma,80000,2014-06-11 9:00:00,Admin,2,3500,2011-06-16 0:00:00
4,3,Vishal,Singhal,300000,2014-02-20 9:00:00,HR,3,4000,2020-02-16 0:00:00


### Many-to-many joins

In [37]:
employee_skills = pd.DataFrame({'group': ['Accounting', 'Accounting',
                                  'Engineering', 'Engineering', 'HR', 'HR'],
                                'skills': ['math', 'spreadsheets', 'coding', 'linux',
                                  'spreadsheets', 'organization']})
employee_skills

,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


#### Merge the employee_group and employee_skill tables

In [38]:
employee_group.merge(employee_skills)

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


## Specification of the Merge Key

In [39]:
employee_salary = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                                'salary': [70000, 80000, 120000, 90000]})
employee_salary

,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


#### Merge the employee_group and employee_hire_date tables and specify the employee column to join on. Assign the resulting table to employees

In [40]:
employees = employee_group.merge(employee_hire_date, on='employee')
employees

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


#### Merge the employees and employee_supervisor tables and specify the group column to join on

In [41]:
employees.merge(employee_supervisor, on='group')

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


#### Merge the employees and employee_salary tables

In [46]:
employees.merge(employee_salary, left_on = 'employee', right_on = 'name')

,employee,group,hire_date,name,salary
0,Bob,Accounting,2008,Bob,70000
1,Jake,Engineering,2012,Jake,80000
2,Lisa,Engineering,2004,Lisa,120000
3,Sue,HR,2014,Sue,90000


___

In [47]:
worker_indexed = worker.set_index('worker_id')
title_indexed = title.set_index('worker_ref_id')

display('worker_indexed', 'title_indexed')

worker_indexed
          first_name last_name  salary        joining_date department
worker_id                                                            
1             Monika     Arora  100000  2014-02-20 9:00:00         HR
2           Niharika     Verma   80000  2014-06-11 9:00:00      Admin
3             Vishal   Singhal  300000  2014-02-20 9:00:00         HR
4             Amitah     Singh  500000  2014-02-20 9:00:00      Admin
5              Vivek     Bhati  500000  2014-06-11 9:00:00      Admin
6              Vipul     Diwan  200000  2014-06-11 9:00:00    Account
7             Satish     Kumar   75000  2014-01-20 9:00:00    Account
8            Geetika   Chauhan   90000  2014-04-11 9:00:00      Admin

title_indexed
                worker_title       affected_from
worker_ref_id                                   
1                    Manager  2016-02-20 0:00:00
2                  Executive  2016-06-11 0:00:00
8                  Executive  2016-06-11 0:00:00
5                    Manager  2016-06-11 0:00:00
4              Asst. Manager  2016-06-11 0:00:00
7                  Executive  2016-06-11 0:00:00
6                       Lead  2016-06-11 0:00:00
3                       Lead  2016-06-11 0:00:00

In [48]:
employee_group_indexed = employee_group.set_index('employee')
employee_group_indexed

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


#### Merge the worker_indexed and title_indexed tables

In [50]:
worker_indexed.merge(title_indexed, left_index=True, right_index=True)

,first_name,last_name,salary,joining_date,department,worker_title,affected_from
1,Monika,Arora,100000,2014-02-20 9:00:00,HR,Manager,2016-02-20 0:00:00
2,Niharika,Verma,80000,2014-06-11 9:00:00,Admin,Executive,2016-06-11 0:00:00
3,Vishal,Singhal,300000,2014-02-20 9:00:00,HR,Lead,2016-06-11 0:00:00
4,Amitah,Singh,500000,2014-02-20 9:00:00,Admin,Asst. Manager,2016-06-11 0:00:00
5,Vivek,Bhati,500000,2014-06-11 9:00:00,Admin,Manager,2016-06-11 0:00:00
6,Vipul,Diwan,200000,2014-06-11 9:00:00,Account,Lead,2016-06-11 0:00:00
7,Satish,Kumar,75000,2014-01-20 9:00:00,Account,Executive,2016-06-11 0:00:00
8,Geetika,Chauhan,90000,2014-04-11 9:00:00,Admin,Executive,2016-06-11 0:00:00


#### Merge the employee_group_indexed and employee_hire_date tables

In [51]:
employee_group_indexed.merge(employee_hire_date, left_index=True, right_on='employee')

,group,employee,hire_date
1,Accounting,Bob,2008
2,Engineering,Jake,2012
0,Engineering,Lisa,2004
3,HR,Sue,2014


## Specifying Set Arithmetic for Joins

In [52]:
food = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
drinks = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])

display('food', 'drinks')

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread
,name,drink
0,Mary,wine
1,Joseph,beer


#### What is the default join type when using the merge method? Add a 'how' parameter with the default join type

- default is `inner`

In [53]:
food.merge(drinks, how = 'inner')

,name,food,drink
0,Mary,bread,wine


#### Merge the food and drinks tables leaving all names from both tables in

In [54]:
food.merge(drinks, how = 'outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


#### Merge food and drinks leaving only rows that have a defined food

In [55]:
food.merge(drinks, how = 'left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


#### Merge food and drinks leaving only row that have a defined drink

In [56]:
food.merge(drinks, how = 'right')

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


## Overlapping Column Names: The ``suffixes`` Keyword

Finally, you may end up in a case where your two input ``DataFrame``s have conflicting column names.
Consider this example:

In [57]:
df1 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df2 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df1', 'df2')

,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4
,name,rank
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


#### Merge df1 and df2 on the name column using the default suffixes

In [58]:
df1.merge(df2, on='name')

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


#### Merge df1 and df2 on the name column again, but this time define a set a suffixes for pandas to use

In [59]:
df1.merge(df2, on='name', suffixes=['_L', '_R'])

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


## Example: US States Data

In [61]:
pop = pd.read_csv('../datasets/state_population.csv')
areas = pd.read_csv('../datasets/state_areas.csv')
abbrevs = pd.read_csv('../datasets/state_abbrevs.csv')
display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state_region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area_sq_mile
0     Alabama         52423
1      Alaska        656425
2     Arizona        114006
3    Arkansas         53182
4  California        163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

### We'll be using merges and joins to calculate statistics about state population. Our ultimate goal is to calculate the state with the largest population density in 2010

#### First, we need to merge our tables together. Merge the pop and abbrevs tables. Drop the redundant abbreviation column

In [78]:
merged = pop.merge(abbrevs, left_on = 'state_region', right_on = 'abbreviation')
merged = merged.drop('abbreviation', 1)
merged.head()

/var/folders/wg/8yz1tny10yl9ct8bq7zl3pp00000gn/T/ipykernel_2725/2782219588.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  merged = merged.drop('abbreviation', 1)


,state_region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


#### It's important to check if the data is valid. Check if there are any nulls in the table

In [79]:
merged.isnull().any()

state_region    False
ages            False
year            False
population      False
state           False
dtype: bool

#### As you can see, there are null values for population and state. Display a table filtered for rows with a null state and one for rows with a null population. Figure out what is wrong with the data using the displayed tables

In [80]:
display("merged[merged['state'].isnull()]", "merged[merged['population'].isnull()]")

,state_region,ages,year,population,state
,state_region,ages,year,population,state


#### The issues are that Puerto Rico and the entire USA area do not exist in the abbrev table and Puerto Rico lacks population data before 2000. In this case, we will drop the missing data because we only care about states. Drop any null rows from merged

In [81]:
merged = merged.dropna()
merged.head()

,state_region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


#### Now you can continue merging tables. Merge the merged and areas tables and store the results in final

In [82]:
final = merged.merge(areas, on='state')
final.head()

,state_region,ages,year,population,state,area_sq_mile
0,AL,under18,2012,1117489.0,Alabama,52423
1,AL,total,2012,4817528.0,Alabama,52423
2,AL,under18,2010,1130966.0,Alabama,52423
3,AL,total,2010,4785570.0,Alabama,52423
4,AL,under18,2011,1125763.0,Alabama,52423


#### Check for nulls in the resulting table

In [83]:
final.isnull().any()

state_region    False
ages            False
year            False
population      False
state           False
area_sq_mile    False
dtype: bool

#### The final table contains no nulls so you can continue. Filter the final table for rows where ages is total and year is 2010. You may use the query method. Store the result in pop_2010 and set the index to state

In [84]:
pop_2010 = final.query("ages == 'total' & year == 2010")
pop_2010.set_index('state', inplace=True)
pop_2010

,state_region,ages,year,population,area_sq_mile
state,,,,,
Alabama,AL,total,2010,4785570.0,52423
Alaska,AK,total,2010,713868.0,656425
Arizona,AZ,total,2010,6408790.0,114006
Arkansas,AR,total,2010,2922280.0,53182
California,CA,total,2010,37333601.0,163707
Colorado,CO,total,2010,5048196.0,104100
Connecticut,CT,total,2010,3579210.0,5544
Delaware,DE,total,2010,899711.0,1954
District of Columbia,DC,total,2010,605125.0,68


#### Now you can calculate the final result. Calculate and create a series 'density' from the columns of pop_2010. Then, sort the values of this series. Print the final result

In [85]:
density = pop_2010['population'] / pop_2010['area_sq_mile']
density.sort_values(ascending=False, inplace=True)
density

state
District of Columbia    8898.897059
Connecticut              645.600649
Maryland                 466.445797
Delaware                 460.445752
Florida                  286.597129
California               228.051342
Illinois                 221.687472
Indiana                  178.197831
Georgia                  163.409902
Hawaii                   124.746707
Kentucky                 107.586994
Alabama                   91.287603
Louisiana                 87.676099
Arizona                   56.214497
Arkansas                  54.948667
Iowa                      54.202751
Colorado                  48.493718
Maine                     37.509990
Kansas                    34.745266
Idaho                     18.794338
Alaska                     1.087509
dtype: float64